In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import matplotlib.pyplot as plt
import mplhep as hep
import awkward as ak
import numpy as np
import uproot
import hist

NanoAODSchema.warn_missing_crossrefs = False
hep.style.use("CMS")

In [ ]:
def ChargeOrdering(LEPTON):
    ''' Sort charge from each event negative first then postive. '''
    index = ak.argsort(LEPTON.charge)
    return LEPTON[index]

def LeptonSelection(DATA, PT, ETA, FLAVOR):
    ''' Event selection: electron antielectron pair, electron.pt > PT, electron.eta > ETA,
    ensures charge conservation and allows for 2 or 4 lepton pairs. '''
    if FLAVOR == 'Electron': x = DATA.Electron
    elif FLAVOR == 'Muon': x = DATA.Muon
    mask_pT_eta = ak.all(x.pt > PT, axis=-1) & ak.all(x.eta > ETA, axis=-1)
    x = x[mask_pT_eta]
    mask_charge = ak.sum(x.charge, axis=-1) == 0
    mask_2e = ak.num(x, axis=-1) == 2
    mask_4e = ak.num(x, axis=-1) == 4
    mask = mask_charge & (mask_2e | mask_4e)
    return x[mask]

def TwoLeptonIM(LEPTON):
    twoLepVec = LEPTON[:,0].add(LEPTON[:,1])
    return twoLepVec.mass

def FourLeptonIM(LEPTON):
    MASS = []
    for i in range(2):
        for j in range(2):
            MASS.append((LEPTON[:,i].add(LEPTON[:,j+2])).mass)
    return MASS

def InvariantMassHist(LEPTON, BINS):
    h = hist.Hist(hist.axis.Regular(BINS, 0, 180, name='Invariant Mass[GeV/C^2]'))
    mask_2l = ak.num(LEPTON, axis=-1) == 2
    mask_4l = ak.num(LEPTON, axis=-1) == 4
    lep2 = LEPTON[mask_2l]
    lep4 = LEPTON[mask_4l]
    h.fill(TwoLeptonIM(lep2))
    for i in range(4):
        h.fill(FourLeptonIM(lep4)[i])
    return h

In [ ]:
fname = "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/ForHiggsTo4Leptons/Run2012B_DoubleElectron.root"

In [ ]:
events = NanoEventsFactory.from_root(
    fname, schemaclass=NanoAODSchema.v6, entry_stop=None
).events()

In [ ]:
selectedLeptons = ChargeOrdering(LeptonSelection(events, 20, 0, 'Electron'))
h1 = InvariantMassHist(selectedLeptons,1000)

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
hep.histplot(h1, label=r'Z $\rightarrow$ 2e')
ax.set_xlim(0, 180)
ax.set_ylabel("Events", fontsize=15)
ax.set_xlabel("$M_{ee} [GeV/C^{2}]$", fontsize=15)
plt.legend(fontsize=15)
plt.tight_layout()
hep.cms.label(rlabel="pT > 20 GeV/C")
plt.savefig("pt20.jpg")
plt.show()